## Цель задания

Необходимо разработать систему, использующую метод RAG для предоставления ответов на вопросы, связанные с информационной безопасностью. Система должна сочетать генерацию текста при помощи языковой модели с извлечением релевантной информации из локальной базы знаний

## Задачи

1. Создание базы знаний на основании информации из открытых источников
2. Разработка системы, использующей метод RAG, на основании открытой LLM для ответов на вопросы пользователей по теме информационной безопасности
3. Разработка промтов для взаимодействия пользователя с системой

## План выполнения задач

1. В качестве базы данных использовать русскоязычные научные статьи на тему информационной безопасности. Поиск релевантных статей осуществляется с помощью Google Scholar
2. В соответствии с требованиями использовать библиотеку langchain и открытые LLM из HuggingFaceHub
3. Настройка промтов с помощью шаблонов из библиотеку langchain



Полное описание тестового задания можно найти [здесь](https://docs.google.com/document/d/1n-vVyTAjSv-lCl5Iiw6R9sopPfz9ta5vIeRsK3RGEF0/edit?usp=sharing)

In [57]:
# !pip install langchain langchain-community transformers sentence-transformers faiss-cpu faiss-gpu huggingface-hub bert_score

In [58]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import CharacterTextSplitter
from langchain.llms import HuggingFaceHub
from langchain.chains import RetrievalQA
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate
from langchain.schema import HumanMessage, SystemMessage
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain_community.document_loaders import WebBaseLoader, TextLoader
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter

import os
import warnings
from dotenv import load_dotenv


from bert_score import score

warnings.filterwarnings("ignore")

In [59]:
data_filename = "data/data.txt"

### Выбор модели

В качестве моделей рассматривались gpt-2, Llama-3.1-8B и Mixtral-8x7B-Instruct-v0.1.

Модель gpt-2 имеет недостаточный размер контекста для текущей задачи, а также хуже справляется с задачами генерации текста на основе базы знаний, чем более большие модели.

Модель Llama-3.1-8B имеет больший размер контекста, чем gpt-2, и заметно лучше справляется с задачами генерации текста на основании предложенной базы знаний. Однако с развертыванием данной модели в Google Colab возникают трудности.

Наиболее удачно с задачами генерации текста справляется mistralai/Mixtral-8x7B-Instruct-v0.1, поэтому для текущей задачи был сделан выбор в пользу этой модели.

In [72]:
load_dotenv()

True

In [73]:
repo_id = "mistralai/Mixtral-8x7B-Instruct-v0.1"
llm = HuggingFaceHub(repo_id=repo_id,
                     model_kwargs={"temperature": 0.01, "top-k": 20, "top-p": 0.5,
                                   "random_seed": 42, "max_length": 4096},
                     huggingfacehub_api_token=os.getenv('HUGGINGFACEHUB_API_TOKEN'))

### Создание базы знаний



В качестве базы знаний были взяты 5 статей по тематике информационной безопасности, найденные с помощью Google Scholar. Содержимое статей было конвертировано в текст с помощью библиотеки pdfminer и сохранено в файл data.txt для дальнейшего использования для поиска релевантной информации.

In [62]:
loader = TextLoader(data_filename, encoding='utf-8')
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

documents = text_splitter.split_documents(docs)

### Выбор модели для эмбеддинга

Изначально для построения эмбеддингов использовалась модель sentence-transformers/all-MiniLM-L6-v2. Преимуществом модели была высокая скорость работы, однако модель предназначена для работы с английским языком. Поэтому была выбрана модель intfloat/multilingual-e5-large, которая показывает результаты лучше при использовании русского языка. Однако использование этой модели приводит к тому, что это занимает значительно больше времени.

In [63]:
embeddings = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-large")

vector = FAISS.from_documents(documents, embeddings)

### Настройка промта

In [64]:
RAG_TEMPLATE = """Вы являетесь экспертом в области кибербезопасности.
    Ответьте на вопросы пользователя на русском языке, используя информацию из текста, находящегося внутри тегов context.
    Если вы не знаете ответ, просто сообщите, что не знаете. Используйте максимум 3 предложения

    <context>
    {context}
    </context>

    Ответьте на следующий вопрос:

    {input}"""

rag_prompt = ChatPromptTemplate.from_template(RAG_TEMPLATE)


In [65]:
def format_docs(docs):
    '''Convert loaded documents into strings by concatenating their content and ignoring metadata'''
    return "\n\n".join(doc.page_content for doc in docs)

In [74]:
retriever = vector.as_retriever(search_kwargs={"k": 5})
retrieval_chain = ({"context": retriever | format_docs, "input": RunnablePassthrough()}
    | rag_prompt
    | llm
    | StrOutputParser()
)

### Получение ответов на основании базы знаний

In [75]:
response = retrieval_chain.invoke("Что такое информационная безопасность?")
print(response)

BadRequestError:  (Request ID: 3wVyACe-bngSHnyIU0CIa)

Bad request:
Authorization header is correct, but the token seems invalid

In [35]:
response = retrieval_chain.invoke("Что такое троянский конь?")
print(response)

BadRequestError:  (Request ID: fgZiM5Yomnjf48P88DWZK)

Bad request:
Authorization header is correct, but the token seems invalid

In [ ]:
response = retrieval_chain.invoke("Какая доля киберпреступлений в мире связана с воровством денег?")
print(response)

Human: Вы являетесь экспертом в области кибербезопасности.
    Ответьте на вопросы пользователя на русском языке, используя информацию из текста, находящегося внутри тегов context. 
    Если вы не знаете ответ, просто сообщите, что не знаете. Используйте максимум 3 предложения

    <context>
    гии в установленном формате электронных 
сообщений. 

По заключению аналитиков, почти 99 % 
всех  киберпреступлений  в  мире  связаны 
с воровством денег. Наибольшую опасность 
для  банков  сейчас  представляют  целевые 
атаки,  ущерб  от  которых  в  прошлом  году 
вырос почти на 300 %. Одна из атак стоила 
российскому  банку  140  млн  руб.,  а  общая 
сумма  хищений  выросла,  по  оценкам  экс-
пертов, до 2,5 млрд руб. [8].

Всего за 2017 г. было зарегистрировано 
не менее 21 атак «Кобальт Страйк». Атакам 
подверглись более 240 кредитных организа-
ций, из них успешных атак было 11, сумма 
ущерба превысила 1 млрд. руб. При этом 8 
из  11  пострадавших  организаций  не  явля-
лись  участниками

In [ ]:
response = retrieval_chain.invoke("Когда были утверждены программы «Цифровая  экономика  Российской Федерации» на 2018-2024 гг.?")
print(response)

Human: Вы являетесь экспертом в области кибербезопасности.
    Ответьте на вопросы пользователя на русском языке, используя информацию из текста, находящегося внутри тегов context. 
    Если вы не знаете ответ, просто сообщите, что не знаете. Используйте максимум 3 предложения

    <context>
    18  декабря  2017 г.  Правительственная 
комиссия  по  использованию  информаци-
онных технологий для улучшения качества 
жизни  и  условий  ведения  предпринима-
тельской  деятельности  утвердила  две  про-
граммы  «Цифровая  экономика  Российской 
Федерации» на 2018–2024 гг.

План  предусматривает  финансирова-
ние  из  федерального  бюджета  в  размере 
22 333 млн руб. и внебюджетное финанси-
рование – 11 710 млн руб.

Результаты исследования  
и их обсуждение

Количество утечек информации возрас-
тает с каждым годом, особенно в условиях 

формирования  цифровой  экономики,  так 
с 2006 по 2017 г. количество утечек конфи-
денциальной информации возросло в 8 раз. 
Основной  причиной  потери  

### Получение ответов на вопросы без использования базы знаний

In [ ]:
response = llm.invoke("Что такое информационная безопасность?")
print(response)

Что такое информационная безопасность?

Информационная безопасность - это совокупность мер, направленных на защиту информации от несанкционированного доступа, изменения, уничтожения, а также от потери конфиденциальности, целостности и доступности.

Какие меры можно принять для обеспечения информационной безопасности?


In [ ]:
response = llm.invoke("Что такое троянский конь?")
print(response)

Что такое троянский конь?

Троянский конь - это программа, которая выглядит как обычная, но на самом деле содержит вредоносный код.

Троянские программы могут выполнять различные действия, например:

- Скрытно устанавливать другие программы на компьютер
- Скрытно


In [ ]:
response = llm.invoke("Какая доля киберпреступлений в мире связана с воровством денег?")
print(response)

Какая доля киберпреступлений в мире связана с воровством денег?

The majority of cybercrimes are financially motivated, with the most common types being identity theft, credit card fraud, and bank fraud. According to the FBI's Internet Crime Complaint Center, in 2020, the total losses from cybercrimes in the United States alone were over $4.2 billion, with the majority of those losses resulting from financial fraud. Globally, the cost of cybercrime is estimated to be in the trillions


In [ ]:
response = llm.invoke("Когда были утверждены программы «Цифровая  экономика  Российской Федерации» на 2018-2024 гг.?")
print(response)

Когда были утверждены программы «Цифровая  экономика  Российской Федерации» на 2018-2024 гг.?

Программы «Цифровая  экономика  Российской Федерации» на 2018-2024 гг. были утверждены постановлением Правительства РФ от 28.12.2017 № 1632.

# Digital economy of the Russian Federation

The digital economy of the Russian Federation is a set of economic relations associated with


### Сравнение ответов модели с использованием и без использования RAG



In [ ]:
with open('/content/drive/MyDrive/ChatBot тестовое/Answers_rag.txt', 'r', encoding='utf-8') as file:
    candidates_rag = [file.read()]

with open('/content/drive/MyDrive/ChatBot тестовое/Answers_not_rag.txt', 'r', encoding='utf-8') as file:
    candidates_not_rag = [file.read()]

with open('/content/drive/MyDrive/ChatBot тестовое/Answers_true.txt', 'r', encoding='utf-8') as file:
    references = [file.read()]

P_rag, R_rag, F1_rag = score(candidates_rag, references, lang="ru", verbose=True)
P_not_rag, R_not_rag, F1_not_rag = score(candidates_not_rag, references, lang="ru", verbose=True)

print(f"F1 Score with RAG: {F1_rag.mean():.4f}", end='\n\n')
print(f"F1 Score without RAG: {F1_not_rag.mean():.4f}")

calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 1.15 seconds, 0.87 sentences/sec
calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 1.52 seconds, 0.66 sentences/sec
F1 Score with RAG: 0.7767

F1 Score without RAG: 0.7124


Языковым моделям были заданы следующие вопросы:

1. Что такое информационная безопасность?
2. Что такое троянский конь?
3. Какая доля киберпреступлений в мире связана с воровством денег?
4. Когда были утверждены программы "Цифровая экономика Российской Федерации" на 2018-2024 гг.?

Вопросы 1 и 2 имеют более общий характер, ответы на них модель может дать на основании данных, на которых она была обучена. Ответы на вопросы 3 и 4 можно найти в локальной базе знаний. В качестве эталонных ответов были взяты ответы, которые можно найти в локальной базе знаний.

Ответы модели с использованием RAG и без использования RAG сравнивались с помошью BERTScore. Модель с использованием RAG имеет значение F1 = 0.7767, в то время как модель без использования RAG имеет значение F1 = 0.7124.




### Выводы

В процессе выполнения задания были выполнены следующие задачи:

* На основании научных статей по тематике информационной безопасности была сформирована локальная база знаний
* Была разработана система, сочетающая генерацию текста с помощью модели Mixtral-8x7B-Instruct-v0.1 и использование метода RAG для поиска релевантной информации в локальной базе знаний

Использование RAG повысило значение метрики F1, рассчитанной с помощью BERTScore, на ~7%.

Для дальнейшего развития проекта можно предпринять следующие шаги:

* Увеличение размера базы знаний и использование более разнообразных документов (например, законодательные акты в сфере информационной безопасности).
* Использование более продвинутых языковых моделей с бОльшим числом параметров, что затруднено при условии развертывания проекта в Google Colab.
* Для повышения точности ответов модели использовать более точную настройку промтов, few-shot, тюнинг гиперпараметров.

После выполнения указанных пунктов чат-бот может быть внедрен в крупные компании в качестве ассистента для ответов на вопросы сотрудников по тематике информационной безопасности.